In [1]:
import pytorch_lightning as pl
import torch
import Operator as op
import utils
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

g_dtype = torch.float32

Using cpu device


Setting up the Hamiltonian

In [4]:
lattice_sites = 3
h1 = op.Sx(0) + op.Sx(1)
op.print_op_list(h1)
h2 = op.Sz(0) + op.Sz(1)
o = op.Sx(1) + []
h1_mat = utils.get_total_mat_els(h1, lattice_sites)
h2_mat = utils.get_total_mat_els(h2, lattice_sites)
o_mat = utils.get_total_mat_els(o, lattice_sites)
h2_range = [(3,4)]

'''
for l in range(lattice_sites):
  hamiltonian = Sx(l)* (h) + Sz(l) * Sz((l+1) % lattice_sites) + hamiltonian
print_op_list(hamiltonian)
'''

adding
adding operator
Hamiltonan = Sx_0 + Sx_1
adding
adding operator
adding
adding to sequence


'\nfor l in range(lattice_sites):\n  hamiltonian = Sx(l)* (h) + Sz(l) * Sz((l+1) % lattice_sites) + hamiltonian\nprint_op_list(hamiltonian)\n'

In [22]:
from torch.utils.data import Dataset, DataLoader
#setting up the datamodule

class spin_data(Dataset):
    def __init__(self, lattice_sites, h_mat_list, h_ranges_list, o_mat, t_min=0, t_max=1):
        
        assert(len(h_mat_list) - 1 == len(h_ranges_list))
        #exact sampling for now
        self.spins = utils.get_all_spin_configs(lattice_sites).type(g_dtype)

        #saving ranges to generate alpha values each batch
        self.t_min = t_min
        self.t_max = t_max
        self.h_ranges_list = h_ranges_list

        #saving mat elements to pass to training loop with respective multipliers each loop
        self.h_mat_list = h_mat_list
        self.o_mat = o_mat
        
    def __len__(self):
        return 1000

    def __getitem__(self, index):
        #creating the random alpha array of numspins with one value of (t, h_ext1, ..)
        alpha_arr = torch.ones((self.spins.shape[0], (len(self.h_mat_list))))
        alpha_arr[:, 0] = ( ( self.t_max - self.t_min ) * torch.rand((1,1)) + self.t_min )
        for i in range( len(self.h_mat_list) - 1 ):
            max = self.h_ranges_list[i][1]
            min = self.h_ranges_list[i][0] 
            alpha_arr[:, i+1] = ( (max - min) * torch.rand((1,1)) + min )
        return self.spins, alpha_arr, torch.cat(), self.o_mat

    def cuda(self):
        self.spins = self.spins.to(device)
        self.alpha_arr = self.alpha_arr.to(device)




In [25]:
data = spin_data(lattice_sites, [h1_mat, h2_mat], h2_range, o_mat)
#data.cuda()
dataloader = DataLoader(dataset=data, batch_size=2)
data_iter = iter(dataloader)
spins, alpha = next(data_iter)
#spins, alpha = data[0]
print(alpha, '\n\n' ,spins)

tensor([[[0.9498, 3.5746],
         [0.9498, 3.5746],
         [0.9498, 3.5746],
         [0.9498, 3.5746],
         [0.9498, 3.5746],
         [0.9498, 3.5746],
         [0.9498, 3.5746],
         [0.9498, 3.5746]],

        [[0.4487, 3.1379],
         [0.4487, 3.1379],
         [0.4487, 3.1379],
         [0.4487, 3.1379],
         [0.4487, 3.1379],
         [0.4487, 3.1379],
         [0.4487, 3.1379],
         [0.4487, 3.1379]]]) 

 tensor([[[-1., -1., -1.],
         [ 1., -1., -1.],
         [-1., -1.,  1.],
         [-1.,  1., -1.],
         [ 1.,  1., -1.],
         [ 1., -1.,  1.],
         [-1.,  1.,  1.],
         [ 1.,  1.,  1.]],

        [[-1., -1., -1.],
         [ 1., -1., -1.],
         [-1., -1.,  1.],
         [-1.,  1., -1.],
         [ 1.,  1., -1.],
         [ 1., -1.,  1.],
         [-1.,  1.,  1.],
         [ 1.,  1.,  1.]]])


In [21]:

a = torch.ones((4,2))
a[:, 0] = ((2 - 1) * torch.rand((1,)) + 1)
print(a)

tensor([[1.8491, 1.0000],
        [1.8491, 1.0000],
        [1.8491, 1.0000],
        [1.8491, 1.0000]])


In [46]:
import torch
from torch.nn import functional as F
import pytorch_lightning as pl
from torch import nn

class Model(pl.LightningModule):

  def __init__(self, lattice_sites, h_mat_list, h_map, o_init_mat, o_init_map):
    '''
    Initializer for neural net
    Parameters
    __________
    lattice_sites: int
    h_mat_list: tensor, dtype=complex
      the matrix elements of the hamiltonian 
      shape = (num_ext_params, )
    '''
    super().__init__()
    self.lattice_net = nn.Sequential(
      nn.Conv1d(1, 8, kernel_size=2, padding=1, padding_mode='circular'),
      nn.ReLU(),
      nn.Conv1d(8, 16, kernel_size=2, padding=1, padding_mode='circular'),
      nn.Flatten(start_dim=1, end_dim=-1)
    )
    
    self.tNN = nn.Sequential(
      nn.Linear(2, 16),
      nn.ReLU(),
      nn.Linear(16, 32),
      nn.ReLU(),
      nn.Linear(32, 64)
    )

    self.psi = nn.Sequential(
      nn.Linear( 64 + 16 * ( lattice_sites + 2 ), 128 ),
      nn.ReLU(),
      nn.Linear(128, 64),
      nn.ReLU(),
      nn.Linear(64,2)
    )

    self.h_mat_list = h_mat_list
    self.o_init_mat = o_init_mat
    self.h_map = h_map
    self.o_init_map = o_init_map

  def forward(self, spins, alpha):
    '''
    Forward function of the neural net to calculate psi. uses same spin config for all alpha values
    Parameters
    __________
    spins: tensor, dtype=float
      tensor of input spins to wave function 
      shape = (num_spin_configs, lattice_sites)
    alpha: tensor, dtype=float
      other inputs to hamiltonian e.g. (time, ext_param) 
      shape = (num_alpha_configs, num_inputs)

    Returns
    _______
    psi: tensor, dtype=complex
      wave function for a combination of (spins, alpha) 
      size = (num_spin_configs, num_alpha_configs)
    '''

    spin_shape = spins.shape
    alpha_shape = alpha.shape
    
    #unsqueeze since circular padding needs tensor of dim 3
    spins = torch.flatten(spins, end_dim=-2).unsqueeze(1)
    alpha = torch.flatten(alpha, end_dim=-2)

    lat_out = self.lattice_net(spins)
    t_out = self.tNN(alpha)

    rad_and_phase = (self.psi(torch.cat((lat_out, t_out), dim=1)))
    psi = rad_and_phase[:, 0] * torch.exp( 1.j * rad_and_phase[:, 1] )
    return psi.reshape( spin_shape[0], spin_shape[1], 1)
    
  
  def training_step(self, batch, batch_idx):
    spins, alpha = batch
    #get psi(s, alpha)
    psi = self(spins, alpha)
    #get map and s' to the s from batch_idx

    #get psi(s', alpha)

    #calc dt_psi(s, alpha)

    #get mat_els for all alphas

    #get map, s' mat_els for O_loc_init

    #calc loss
    print('nyi')

  def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

model = Model(lattice_sites)
print(model)
#print(model(spins, alpha))

#trainer = pl.Trainer(fast_dev_run=True)
#trainer.fit(model, dataloader)


SyntaxError: invalid syntax (<ipython-input-46-a53fa7ac6cbc>, line 67)

In [6]:
from timeit import default_timer as timer

map = utils.get_map(hamiltonian, lattice_sites)
map = map.to(device)
print("map: ", map)

mat_els = utils.get_total_mat_els(hamiltonian, lattice_sites)
mat_els = mat_els.to(device)
print("mat els: ", mat_els)

#1.dim: Batch
#2.dim: lattice sites
#s_config = get_all_spin_configs(lattice_sites)
s_config = torch.tensor([1,1,1]).reshape(1,3)
s_config = (s_config.type(torch.float32)).to(device)
print("spin config: ", s_config)

start = timer()
s_p = utils.get_sp(s_config, map)
psi_s = model(s_config)
psi_sp = model(s_p.reshape(-1, lattice_sites)).reshape(s_p.shape[0], s_p.shape[1])
print(psi_sp.shape, psi_s.shape, s_config.shape)
O_loc = utils.calc_Oloc(psi_sp, mat_els, s_config)
end = timer()

print(f"time to calculate O_loc: {end - start:.2e}") 

map:  tensor([[[ 1,  1, -1],
         [ 1,  1, -1]]], dtype=torch.int8)
mat els:  tensor([[[[ 1.+0.j,  1.+0.j,  1.+0.j],
          [ 1.+0.j,  1.+0.j,  0.+1.j]],

         [[ 1.+0.j,  1.+0.j,  1.+0.j],
          [ 1.+0.j, -1.+0.j, -0.-1.j]]]])
spin config:  tensor([[1., 1., 1.]])


TypeError: forward() missing 1 required positional argument: 'alpha'

In [9]:
spins = torch.full((2,5), 0.5)
spins[1,:] /=4

alpha = torch.full((4,2), 1)
alpha[:, 1] = 2
alpha[:2, : ]*=4
spins_batched = torch.flatten(torch.broadcast_to(spins, (alpha.shape[0], spins.shape[0], spins.shape[1])), end_dim=-2)
alpha_batched = torch.flatten(torch.broadcast_to(alpha.unsqueeze(1), (alpha.shape[0], spins.shape[0], alpha.shape[1])), end_dim=-2)
print(alpha)
print(spins)

print(spins_batched)
print(alpha_batched)
#torch.cat((alpha, spins), dim=2)

tensor([[4, 8],
        [4, 8],
        [1, 2],
        [1, 2]])
tensor([[0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
tensor([[0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
tensor([[4, 8],
        [4, 8],
        [4, 8],
        [4, 8],
        [1, 2],
        [1, 2],
        [1, 2],
        [1, 2]])


In [48]:
a = torch.arange(0,8).reshape(2,2,2)
print(a)
a = torch.flatten(a, end_dim=-2)
print(a)
a = a.reshape(2,2,2)
print(a)


tensor([[[0, 1],
         [2, 3]],

        [[4, 5],
         [6, 7]]])
tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7]])
tensor([[[0, 1],
         [2, 3]],

        [[4, 5],
         [6, 7]]])
